## Train Multiple LIB Models

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk, concatenate_datasets
from sklearn.metrics import accuracy_score
import random
from tqdm import tqdm
import numpy as np
import torch
import os

/home/jschelb/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set Random Seed for Reproducibility

In [2]:
# Set a seed for random module
random.seed(42)

# Set a seed for numpy module
np.random.seed(42)

# Set a seed for torch module
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Define Parameters

In [3]:
SAMPLING = "random" # "random", "stratified", "clustered", "shared_domain"
SUFFIX = "_extended" #"", "_holdout", "_extended"
SPLIT = "extended" # "train", "test", "holdout", "extende
MAX_CONTENT_LENGTH = 384 # 496, 192
OVERLAP = 64
FEATURES = "url" # "url", "content", "url_and_content"

In [4]:
TOPICS = ["cannabis", "kinder", "energie"]
#TOPICS = ["cannabis"]

**Extract URL-path:**

In [5]:
from urllib.parse import urlparse, urlunparse

def extract_url_path(example):
    view_url = example['view_url']
    if "://" not in view_url:
        view_url = "http://" + view_url  # Assume http if no protocol specified
    parsed_url = urlparse(view_url)
    new_url = urlunparse(('', '', parsed_url.path, parsed_url.params, parsed_url.query, parsed_url.fragment))
    example['url_path'] = new_url.lstrip('/')  # Store the result in a new field
    return example


extract_url_path({"view_url": "https://www.google.com/search?q=python+url+path"})

{'view_url': 'https://www.google.com/search?q=python+url+path',
 'url_path': 'search?q=python+url+path'}

## Define Model

In [6]:
import numpy as np
from collections import defaultdict, Counter
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import normalize

In [7]:
################################# Extract n-grams from a URL #################################

def extract_ngrams(url, n):
    """Extracts n-grams of length n from the given URL."""
    return [url[i:i+n] for i in range(len(url) - n + 1)]

################################# Calculate frequencies of n-grams #################################

def calculate_ngram_frequencies(urls, genres, n_range):
    """Calculates the frequency of n-grams for each genre from the given URLs and genres."""
    
    ngram_freq = defaultdict(lambda: defaultdict(int))
    genre_counts = Counter(genres)
    
    for url, genre in zip(urls, genres):
        for n in n_range:
            ngrams = extract_ngrams(url, n)
            for ngram in ngrams:
                ngram_freq[genre][ngram] += 1
    
    return ngram_freq, genre_counts

################################# Calculate the mutual information for n-grams #################################

def calculate_mutual_information(ngram_freq, genre_counts):
    """Calculates mutual information for n-grams using their frequencies and genre counts."""
    
    ngrams = list({ngram for genre in ngram_freq for ngram in ngram_freq[genre]})
    X = np.zeros((len(ngram_freq), len(ngrams)))
    y = list(genre_counts.keys())

    for i, genre in enumerate(y):
        for j, ngram in enumerate(ngrams):
            X[i, j] = ngram_freq[genre][ngram]
    
    mi = mutual_info_classif(X, y, discrete_features=True)
    return dict(zip(ngrams, mi))

################################# Estimate the interpolation coefficients #################################

def estimate_interpolation_coefficients(ngram_mi, n_range):
    """Estimates interpolation coefficients based on mutual information of n-grams."""
    
    lambda_sum = sum(ngram_mi.values())
    lambdas = {n: 0 for n in n_range}
    
    for ngram, mi in ngram_mi.items():
        n = len(ngram)
        lambdas[n] += mi / lambda_sum
    
    # Normalize the coefficients
    total = sum(lambdas.values())
    for n in lambdas:
        lambdas[n] /= total
    
    return lambdas

################################# Main training function #################################

def train_genre_classifier(urls, genres, n_range=(2, 3, 4)):
    """
    Trains the genre classifier by calculating n-gram frequencies, mutual information,
    and interpolation coefficients from the given URLs and genres.
    """
    # Calculate n-gram frequencies and genre counts
    ngram_freq, genre_counts = calculate_ngram_frequencies(urls, genres, n_range)
    
    # Calculate mutual information for n-grams
    ngram_mi = calculate_mutual_information(ngram_freq, genre_counts)
    
    # Estimate interpolation coefficients
    lambdas = estimate_interpolation_coefficients(ngram_mi, n_range)
    
    return ngram_freq, genre_counts, lambdas

In [8]:
################################# Normalize the probabilities #################################

def normalize_probs(probs):
    """Normalizes the probability dictionary so that the probabilities sum to 1."""
    total = sum(probs.values())
    if total == 0:
        return {k: 0 for k in probs}  # Return zero probabilities if total is zero
    return {k: v / total for k, v in probs.items()}

################################# Calculate the probability of an n-gram for a given genre #################################

def calculate_ngram_prob(ngram, genre, ngram_freq, genre_counts, lambdas):
    """
    Calculates the probability of an n-gram given a genre using pre-trained frequencies and interpolation coefficients.
    """
    n = len(ngram)
    lambda_n = lambdas.get(n, 0)
    freq = ngram_freq[genre].get(ngram, 0)
    total_ngrams = sum(ngram_freq[genre].values())
    if total_ngrams == 0:
        return 1e-10  # Return a small non-zero value if no n-grams are found
    prob = (lambda_n * (freq / total_ngrams))
    return prob

################################# Extract n-grams from a URL #################################

def extract_ngrams(url, n):
    """Extracts n-grams of length n from the given URL."""
    return [url[i:i+n] for i in range(len(url) - n + 1)]

################################# Prediction function #################################

def predict_genre(url, ngram_freq, genre_counts, lambdas, n_range=(2, 3, 4)):
    """
    Predicts the genre of a given URL using pre-trained n-gram frequencies, genre counts, and interpolation coefficients.
    """
    probs = {genre: np.log(count / sum(genre_counts.values())) if count > 0 else -np.inf for genre, count in genre_counts.items()}
    Q = []
    grams = set()
    
    for n in n_range:
        Q.extend(extract_ngrams(url, n))
    
    while Q:
        gram = Q.pop(0)
        if any(gram in ngram_freq[genre] for genre in genre_counts):
            grams.add(gram)
        else:
            if len(gram) > min(n_range):
                Q.append(gram[:-1])
    
    if grams:
        for genre in genre_counts:
            for gram in grams:
                ngram_prob = calculate_ngram_prob(gram, genre, ngram_freq, genre_counts, lambdas)
                if ngram_prob > 0:
                    probs[genre] += np.log(ngram_prob)
    
    probs = normalize_probs(probs)
    predicted_genre = max(probs, key=probs.get)
    
    return predicted_genre, probs


## Train Model

In [9]:
def prepare_input_data(train_data, features):
    """Prepares input data based on the specified features."""
    X_train = []
    for row in train_data:
        if features == "content":
            input = row["text"]
        elif features == "url":
            input = row["url_path"]
        elif features == "url_and_content":
            input = row["url_path"] + " " + row["text"]
        else:
            raise ValueError("Invalid value for features. Expected 'content', 'url', or 'url_and_content'.")
        X_train.append(input)
    return X_train

## Evaluate Models

In [10]:
def get_predictions(tokenized_datasets, features, ngram_freq, genre_counts, lambdas, n_range, split="test"):
    """Use the trained model to make predictions on the test set."""
    
    preds = []
    labels = []
    input_data = prepare_input_data(tokenized_datasets[split], features)
    
    for i, input in enumerate(tqdm(input_data)):
        predicted_class, _ = predict_genre(input, ngram_freq, genre_counts, lambdas, n_range)
        preds.append(predicted_class)
        labels.append(tokenized_datasets[split][i]["label"])
    
    return preds, labels

In [11]:
def calc_metrics(labels, preds):
    """
    Calculates the accuracy, precision, recall, and F1 score for the given labels and predictions and returns them in a dictionary.
    """
    
    metrics = {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, average='binary'),
        'recall': recall_score(labels, preds, average='binary'),
        'f1': f1_score(labels, preds, average='binary'),
    }

    return metrics

In [12]:
def sample_random_from_dataset(dataset, n=5, subset='test'):
    """
    Samples n random examples from a specified subset of the dataset.
    """
    n = min(n, len(dataset[subset]))
    random_indices = random.sample(range(len(dataset[subset])), n)
    sampled_dataset = dataset[subset].select(random_indices)
    return sampled_dataset

**Get chunk level predictions:**

In [13]:
from collections import defaultdict
eval_results = defaultdict(dict)

for topic in TOPICS: # ----------------------------------------------------------------------

    print(f"\n\n###### Evaluating model on {topic} ###### \n\n")
    
    if FEATURES == "url":
        dataset = load_from_disk(f"../../data/tmp/processed_dataset_{topic}_buffed_{SAMPLING}{SUFFIX}")
        
        if SPLIT == "holdout":
                dataset["holdout"] = concatenate_datasets(
                    [dataset["holdout"], dataset["test"]])
        dataset = dataset.map(extract_url_path, num_proc=8) # Extract the path from the URL
    else:
        dataset = load_from_disk(f"../../data/tmp/processed_dataset_{topic}_buffed_chunkified_{SAMPLING}{SUFFIX}_{MAX_CONTENT_LENGTH}")
        
        if SPLIT == "holdout":
                dataset["holdout"] = concatenate_datasets(
                    [dataset["holdout"], dataset["test"]])
                
        dataset = dataset.map(extract_url_path, num_proc=8) # Extract the path from the URL

    # Train the model
    X_train = prepare_input_data(dataset['train'], FEATURES)
    y_labels = dataset["train"]["label"]
    n_range = (2, 3, 4)
    ngram_freq, genre_counts, lambdas = train_genre_classifier(X_train, y_labels, n_range)
    print(f"Trained model for {topic}")
    
    # Use the trained model to make predictions on the test set
    preds, labels = get_predictions(dataset, FEATURES, ngram_freq, genre_counts, lambdas, n_range, split=SPLIT)
    metrics = calc_metrics(labels, preds)
    print(f"Metrics for {topic}: {metrics}")
    
    # Add answers to the dataset
    dataset[SPLIT] = dataset[SPLIT].add_column("preds", preds)
    dataset.save_to_disk(f"../../data/tmp/processed_dataset_{topic}_buffed_chunkified_{SAMPLING}{SUFFIX}_{MAX_CONTENT_LENGTH}_s_LIB_{FEATURES}_{SPLIT}")
    
    # Update the eval_results dictionary
    eval_results[topic] = metrics



###### Evaluating model on cannabis ###### 


Trained model for cannabis


 13%|█▎        | 5600/44432 [05:06<35:23, 18.28it/s]  


KeyboardInterrupt: 

In [ ]:
print(eval_results)

defaultdict(<class 'dict'>, {'cannabis': {'accuracy': 0.9565217391304348, 'precision': 0.9565217391304348, 'recall': 0.9565217391304348, 'f1': 0.9565217391304348}, 'kinder': {'accuracy': 0.8636363636363636, 'precision': 0.9444444444444444, 'recall': 0.7727272727272727, 'f1': 0.85}, 'energie': {'accuracy': 0.8695652173913043, 'precision': 0.9047619047619048, 'recall': 0.8260869565217391, 'f1': 0.8636363636363636}})


### Save Chunk Level Predictions and Output Results

In [ ]:
from IPython.display import display, HTML
from tabulate import tabulate
import json

In [ ]:
# Define the file path to save the dictionary
file_path =f"eval_results_lib_{FEATURES}_{SPLIT}_chunks.json"

In [ ]:
# Save the dictionary to disk as JSON
with open(file_path, "w") as file:
    json.dump(eval_results, file)

In [ ]:
with open(file_path, "r") as file:
    eval_results = json.load(file)

## Page Level Predictions

In [ ]:
from collections import Counter

In [ ]:
def majority_voting(answers):
    """Apply majority voting to a list of arbitrary classification answers."""
    count = Counter(answers)
    most_common = count.most_common()  # Get all common answers sorted by frequency

    if not most_common:
        return 0 # Handle empty input scenario

    # Check for ties at the highest count
    max_votes = most_common[0][1]
    tied_classes = [cls for cls, votes in most_common if votes == max_votes]

    if len(tied_classes) > 1:
        return max(tied_classes)  # Return the maximum class label in case of a tie
    return tied_classes[0]  # Return the class with the most votes

majority_voting([1, 1, 2, 2, 2, 3])

2

In [ ]:
from collections import defaultdict
eval_results_pages = defaultdict(dict)

for topic in TOPICS: # ----------------------------------------------------------------------

    print(f"\n\n###### Evaluating on {topic} ###### \n\n")
    dataset = load_from_disk(f"../../data/tmp/processed_dataset_{topic}_buffed_chunkified_{SAMPLING}{SUFFIX}_{MAX_CONTENT_LENGTH}_s_LIB_{FEATURES}_{SPLIT}")
    
    print(dataset)
    
    # Group dataset examples by URL, with a fallback to domain
    grouped_dataset = {}
    for example in tqdm(dataset[SPLIT]):
        url = example.get("view_url") or example.get("domain")
        example_filtered = {k: example[k] for k in ["text", "domain", "preds", "label", "category", "annotation_type", "lang"]}
        grouped_dataset.setdefault(url, []).append(example_filtered)
        
    # Extract labels
    labels = []
    for url, chunks in grouped_dataset.items():
        preds = [chunk["label"] for chunk in chunks]
        labels.append(max(preds))
        
    # Merge chunk level predictions
    predictions = []
    for url, chunks in grouped_dataset.items():
        preds = [chunk["preds"] for chunk in chunks]
        pred = majority_voting([pred for pred in preds if pred > 0]) if max(preds) > 0 else 0
        predictions.append(pred)

    # Use the trained model to make predictions on the test set
    metrics = calc_metrics(labels, predictions)
    print(f"Metrics for {topic}: {metrics}")
    
    # Update the eval_results dictionary
    eval_results_pages[topic] = metrics
    



###### Evaluating on cannabis ###### 


DatasetDict({
    train: Dataset({
        features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'token_count', 'url_path'],
        num_rows: 410
    })
    test: Dataset({
        features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'token_count', 'url_path', 'preds'],
        num_rows: 46
    })
})


100%|██████████| 46/46 [00:00<00:00, 3522.89it/s]


Metrics for cannabis: {'accuracy': 0.9534883720930233, 'precision': 0.95, 'recall': 0.95, 'f1': 0.95}


###### Evaluating on kinder ###### 


DatasetDict({
    train: Dataset({
        features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'token_count', 'url_path'],
        num_rows: 384
    })
    test: Dataset({
        features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'token_count', 'url_path', 'preds'],
        num_rows: 44
    })
})


100%|██████████| 44/44 [00:00<00:00, 3509.74it/s]


Metrics for kinder: {'accuracy': 0.8837209302325582, 'precision': 0.9444444444444444, 'recall': 0.8095238095238095, 'f1': 0.8717948717948718}


###### Evaluating on energie ###### 


DatasetDict({
    train: Dataset({
        features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'token_count', 'url_path'],
        num_rows: 408
    })
    test: Dataset({
        features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'token_count', 'url_path', 'preds'],
        num_rows: 46
    })
})


100%|██████████| 46/46 [00:00<00:00, 4048.81it/s]

Metrics for energie: {'accuracy': 0.8695652173913043, 'precision': 0.9047619047619048, 'recall': 0.8260869565217391, 'f1': 0.8636363636363636}


In [ ]:
print(eval_results_pages)

defaultdict(<class 'dict'>, {'cannabis': {'accuracy': 0.9534883720930233, 'precision': 0.95, 'recall': 0.95, 'f1': 0.95}, 'kinder': {'accuracy': 0.8837209302325582, 'precision': 0.9444444444444444, 'recall': 0.8095238095238095, 'f1': 0.8717948717948718}, 'energie': {'accuracy': 0.8695652173913043, 'precision': 0.9047619047619048, 'recall': 0.8260869565217391, 'f1': 0.8636363636363636}})


### Save Chunk Level Predictions and Output Results

In [ ]:
# Define the file path to save the dictionary
file_path = f"eval_results_lib_{FEATURES}_{SPLIT}_pages.json"

In [ ]:
# Save the dictionary to disk as JSON
with open(file_path, "w") as file:
    json.dump(eval_results_pages, file)

In [ ]:
with open(file_path, "r") as file:
    eval_results_pages = json.load(file)